## Импорты

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics, tree
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              GradientBoostingClassifier,
                              RandomForestClassifier)
from sklearn.feature_extraction.text import (CountVectorizer, TfidfTransformer,
                                             TfidfVectorizer)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from transformers import AutoTokenizer, T5ForConditionalGeneration

d:\Kirill\Desktop\Project\App\project\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Скоринги

In [2]:
def my_custom_scorer_f1(y_test, predicted):
    return metrics.classification_report(y_test, predicted, output_dict=True)["Содержательные"][
        "f1-score"
    ]


def my_custom_scorer_precision(y_test, predicted):
    return metrics.classification_report(y_test, predicted, output_dict=True)["Содержательные"][
        "precision"
    ]


def my_custom_scorer_recall(y_test, predicted):
    return metrics.classification_report(y_test, predicted, output_dict=True)["Содержательные"][
        "recall"
    ]


scoring = {
    "f1": metrics.make_scorer(my_custom_scorer_f1, greater_is_better=True),
    "precision": metrics.make_scorer(
        my_custom_scorer_precision, greater_is_better=True
    ),
    "recall": metrics.make_scorer(my_custom_scorer_recall, greater_is_better=True),
    "accuracy": metrics.make_scorer(metrics.accuracy_score),
}

## Датасет

In [3]:
df = pd.read_json("./data/reviews_texts.json")

In [4]:
df.head()

,text,full_sentence,class,lemmas
0,"Очень понравился, .","Очень понравился, заказала 2ой.",Бессодержательные,"[очень, понравиться]"
1,заказала 2 ой,"Очень понравился, заказала 2ой.",Бессодержательные,"[заказать, ой]"
2,", только вниз положила камень для центра тяжести,","Класс, только вниз положила камень для центра ...",Содержательные,"[вниз, положить, камень, центр, тяжесть]"
3,"Очень лёгкое, верх из за цветка тяжёлый","Класс, только вниз положила камень для центра ...",Содержательные,"[очень, легкий, верх, цветок, тяжелый]"
4,Класс получился.,"Класс, только вниз положила камень для центра ...",Бессодержательные,"[класс, получиться]"


## Тональность

In [5]:
df["full_sentence"]

0                         Очень понравился, заказала 2ой.
1                         Очень понравился, заказала 2ой.
2       Класс, только вниз положила камень для центра ...
3       Класс, только вниз положила камень для центра ...
4       Класс, только вниз положила камень для центра ...
                              ...                        
2985    Кашпо действительно отличное. Было  надёжно уп...
2986                                        Отличная ваза
2987        Высокий. Очень легкий. Пришел без повреждений
2988        Высокий. Очень легкий. Пришел без повреждений
2989        Высокий. Очень легкий. Пришел без повреждений
Name: full_sentence, Length: 2990, dtype: object

In [6]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

messages = df["text"]

results = model.predict(messages, k=-1)
df["tone"] = [max(t, key=t.get) for t in results]
df["sentiment"] = results



In [7]:
["positive", "negative"]

['positive', 'negative']

In [8]:
temp_arr = []
for tones in results:
    p = tones["positive"]
    n = tones["negative"]
    if p > n:
        temp_arr.append("positive")
    else:
        temp_arr.append("negative")
temp_arr

['positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',

In [9]:
[(t["positive"], t["negative"]) for t in results]

[(0.9790457487106323, 0.0027249858248978853),
 (1.0000003385357559e-05, 0.006913838442414999),
 (0.01972912810742855, 0.026769286021590233),
 (0.320831298828125, 0.00942259095609188),
 (0.9615437984466553, 1.0000003385357559e-05),
 (1.0000003385357559e-05, 0.08510905504226685),
 (0.2509227991104126, 0.0005803157691843808),
 (0.9740526676177979, 0.0007196703809313476),
 (0.09010298550128937, 0.05034062638878822),
 (1.0000003385357559e-05, 1.0000003385357559e-05),
 (0.07160120457410812, 0.1067005917429924),
 (0.001108730910345912, 0.0009210511343553662),
 (0.9990989565849304, 1.0000003385357559e-05),
 (0.01972912810742855, 0.010023568756878376),
 (1.0000003385357559e-05, 0.0011795104946941137),
 (0.007355827372521162, 0.5775054097175598),
 (0.016924913972616196, 0.1968362182378769),
 (0.9994572401046753, 1.0000003385357559e-05),
 (0.1824355274438858, 1.0000003385357559e-05),
 (0.8596737384796143, 0.004765199031680822),
 (1.0000003385357559e-05, 1.0000003385357559e-05),
 (0.01451358292251

In [10]:
[max(t, key=t.get) for t in results]

['positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'speech',
 'neutral',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'positive',
 'speech',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'speech',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'speech',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 

In [11]:
[max(tones, key=tones.get) for tones in results for t in tones if t in ["positive", "negative"]]

['positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'speech',
 'speech',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',


In [12]:
df["sentiment"].apply(pd.Series).sum()

C:\Users\Kirill\AppData\Local\Temp\ipykernel_35304\3458693904.py:1: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df["sentiment"].apply(pd.Series).sum()


positive    1000.710319
neutral     1243.581681
negative     181.948663
skip         176.785263
speech       232.426127
dtype: float64

## Модель

In [13]:
## Создаём X и y
X = df
y = df["class"].values

# Разбиение на обучающий и тестовый наборы - TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=110
)

In [14]:
param_grid = {"clf__n_estimators": [60], "clf__max_depth": [9]}

In [15]:
base_model = Pipeline(
    [
        ("tfidf", TfidfVectorizer()),
        ("clf", GradientBoostingClassifier()),
    ]
    )

In [16]:
grid = GridSearchCV(estimator=base_model,
                          param_grid=param_grid,
                          cv=5,
                          scoring=["accuracy"],
                          return_train_score=True,
                          n_jobs=-1,
                          refit="accuracy")

In [17]:
grid.fit(X_train["lemmas"].str.join(" ").values, y_train)
predictions = grid.predict(X_test["lemmas"].str.join(" ").values)

In [18]:
print(metrics.classification_report(y_test, predictions))

                   precision    recall  f1-score   support

Бессодержательные       0.88      0.93      0.91       649
         Доставка       0.74      0.65      0.69        60
   Содержательные       0.77      0.66      0.71       188

         accuracy                           0.85       897
        macro avg       0.80      0.75      0.77       897
     weighted avg       0.85      0.85      0.85       897



In [53]:
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.tree import plot_tree
from sklearn.tree import export_graphviz

def report_model(model):
    model_preds = model.predict(X_test["lemmas"].str.join(" ").values)
    print(classification_report(y_test,model_preds))
    print('\n')
    plt.figure(figsize=(12,8),dpi=150)
    plot_tree(model[-1],filled=True,feature_names=X.columns);

In [52]:
report_model(grid.best_estimator_)

                   precision    recall  f1-score   support

Бессодержательные       0.88      0.93      0.91       649
         Доставка       0.74      0.65      0.69        60
   Содержательные       0.77      0.66      0.71       188

         accuracy                           0.85       897
        macro avg       0.80      0.75      0.77       897
     weighted avg       0.85      0.85      0.85       897





InvalidParameterError: The 'decision_tree' parameter of plot_tree must be an instance of 'sklearn.tree._classes.DecisionTreeClassifier' or an instance of 'sklearn.tree._classes.DecisionTreeRegressor'. Got GradientBoostingClassifier(max_depth=9, n_estimators=60) instead.

<Figure size 1800x1200 with 0 Axes>

In [19]:
# import pickle
# filepath = "./StreamLit/model.sav"
# pickle.dump(grid, open(filepath, 'wb'))

In [20]:
# loaded_model = pickle.load(open(filepath, 'rb'))
loaded_model = grid

In [21]:
# X_test.to_json("./StreamLit/data/test_data.json",orient="records",force_ascii=False)

In [22]:
X_test["prediction"] = predictions

In [23]:
probs = loaded_model.predict_proba(X_test["lemmas"].str.join(" ").values)
probs

array([[0.92233598, 0.00172562, 0.0759384 ],
       [0.92233598, 0.00172562, 0.0759384 ],
       [0.92233598, 0.00172562, 0.0759384 ],
       ...,
       [0.92233598, 0.00172562, 0.0759384 ],
       [0.92233598, 0.00172562, 0.0759384 ],
       [0.60258402, 0.00222758, 0.3951884 ]])

In [24]:
X_test["proba"] = [x[2] for x in probs]

In [25]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 10)

In [26]:
df_usefull = X_test[X_test["prediction"]=="Содержательные"]

In [27]:
df_usefull.columns.tolist()

['text',
 'full_sentence',
 'class',
 'lemmas',
 'tone',
 'sentiment',
 'prediction',
 'proba']

In [28]:
cols = ['text', 'full_sentence', 'lemmas', 'class', 'prediction', 'proba']
df_usefull = df_usefull[cols]

In [29]:
df_usefull.sort_values(by=['proba'], ascending=False).head(10)

,text,full_sentence,lemmas,class,prediction,proba
1511,дно было был не устойчив кривое качался отказ!,дно было кривое и вазон качался был не устойчи...,"[дно, устойчивый, кривой, качаться, отказ]",Содержательные,Содержательные,0.999738
1694,"Пластик горшок вес 1 кг, если будет хлипкий! ,...","Пластик хлипкий,горшок внутренний без отверсти...","[пластик, горшок, вес, хлипкий, внутренний, от...",Содержательные,Содержательные,0.992018
1814,Лёгкий пластик легко опрокинет.,"Если ищете кашпо идеально белого цвета, то это...","[легкий, пластик, легко, опрокинуть]",Содержательные,Содержательные,0.991231
1226,Дизайн кашпо хорошо вписался в интерьер комнаты.,"Отличное кашпо, радует глаз. Дизайн кашпо хоро...","[дизайн, кашпо, вписаться, интерьер, комната]",Содержательные,Содержательные,0.989373
15,", стоит неустойчиво т. к. неровное дно",Кашпо сделано из очень мягкого и тонкого мате...,"[стоить, неустойчиво, т, неровный, дно]",Содержательные,Содержательные,0.986126
2512,В подарок положили мешочек с землей.,"Кашпо отличное, по размеру такое как и хотелос...","[подарок, положить, мешочек, земля]",Содержательные,Содержательные,0.981630
1030,Очень симпатично смотрится в интерьере.,Очень симпатично смотрится в интерьере. Упаков...,"[очень, симпатично, смотреться, интерьер]",Содержательные,Содержательные,0.981497
1116,"Дешёвый пластик, как кашпо,","Дешёвый и тонкий пластик, как кашпо ,очень оче...","[дешевый, пластик, кашпо]",Содержательные,Содержательные,0.968775
2589,с приятным бонусом - грунт еще и в подарок,Отличное кашпо! Качество на уровне! Пришло в к...,"[приятный, бонус, грунт, подарок]",Содержательные,Содержательные,0.967387
1728,"Прекрасный продавец, положил в подарок большой...","Отличное кашпо! Прекрасный продавец, положил в...","[прекрасный, продавец, положить, подарок, боль...",Бессодержательные,Содержательные,0.966655


In [30]:
df_usefull[df_usefull["text"] == "Стоит неустойчиво т. к. неровное дно"]

,text,full_sentence,lemmas,class,prediction,proba


In [31]:
df_usefull["text"] = df_usefull["text"].replace(
    to_replace=", стоит неустойчиво т. к. неровное дно",
    value="Стоит неустойчиво т. к. неровное дно",
)
df_usefull["text"] = df_usefull["text"].replace(
    to_replace="дно было был не устойчив кривое качался отказ!",
    value="Дно было не устойчиво кривое качался отказ!",
)

In [32]:
df_usefull.sort_values(by=['proba'], ascending=False).head(10)

,text,full_sentence,lemmas,class,prediction,proba
1511,Дно было не устойчиво кривое качался отказ!,дно было кривое и вазон качался был не устойчи...,"[дно, устойчивый, кривой, качаться, отказ]",Содержательные,Содержательные,0.999738
1694,"Пластик горшок вес 1 кг, если будет хлипкий! ,...","Пластик хлипкий,горшок внутренний без отверсти...","[пластик, горшок, вес, хлипкий, внутренний, от...",Содержательные,Содержательные,0.992018
1814,Лёгкий пластик легко опрокинет.,"Если ищете кашпо идеально белого цвета, то это...","[легкий, пластик, легко, опрокинуть]",Содержательные,Содержательные,0.991231
1226,Дизайн кашпо хорошо вписался в интерьер комнаты.,"Отличное кашпо, радует глаз. Дизайн кашпо хоро...","[дизайн, кашпо, вписаться, интерьер, комната]",Содержательные,Содержательные,0.989373
15,Стоит неустойчиво т. к. неровное дно,Кашпо сделано из очень мягкого и тонкого мате...,"[стоить, неустойчиво, т, неровный, дно]",Содержательные,Содержательные,0.986126
2512,В подарок положили мешочек с землей.,"Кашпо отличное, по размеру такое как и хотелос...","[подарок, положить, мешочек, земля]",Содержательные,Содержательные,0.981630
1030,Очень симпатично смотрится в интерьере.,Очень симпатично смотрится в интерьере. Упаков...,"[очень, симпатично, смотреться, интерьер]",Содержательные,Содержательные,0.981497
1116,"Дешёвый пластик, как кашпо,","Дешёвый и тонкий пластик, как кашпо ,очень оче...","[дешевый, пластик, кашпо]",Содержательные,Содержательные,0.968775
2589,с приятным бонусом - грунт еще и в подарок,Отличное кашпо! Качество на уровне! Пришло в к...,"[приятный, бонус, грунт, подарок]",Содержательные,Содержательные,0.967387
1728,"Прекрасный продавец, положил в подарок большой...","Отличное кашпо! Прекрасный продавец, положил в...","[прекрасный, продавец, положить, подарок, боль...",Бессодержательные,Содержательные,0.966655


In [33]:
from collections import Counter
to_count = np.concatenate(df_usefull["lemmas"].to_numpy(), axis=None)
counter = Counter(to_count)
counter.most_common()

[('подарок', 46),
 ('интерьер', 26),
 ('грунт', 22),
 ('кашпо', 21),
 ('цена', 18),
 ('очень', 16),
 ('положить', 15),
 ('земля', 14),
 ('вписаться', 13),
 ('дно', 12),
 ('приятный', 8),
 ('устойчивый', 8),
 ('стоить', 7),
 ('большой', 7),
 ('пластик', 7),
 ('смотреться', 6),
 ('горшок', 6),
 ('отлично', 6),
 ('высокий', 6),
 ('спасибо', 6),
 ('пакет', 6),
 ('отличный', 5),
 ('ручка', 5),
 ('легкий', 5),
 ('немного', 4),
 ('сделать', 4),
 ('отверстие', 4),
 ('внутренний', 4),
 ('красиво', 4),
 ('просто', 4),
 ('дешевый', 4),
 ('хороший', 4),
 ('удивить', 4),
 ('коробка', 3),
 ('неустойчивый', 3),
 ('заказывать', 3),
 ('цвет', 3),
 ('дизайн', 3),
 ('довольный', 3),
 ('удобный', 3),
 ('выпуклый', 3),
 ('обещать', 3),
 ('особенно', 3),
 ('подарочек', 3),
 ('такой', 3),
 ('вписываться', 3),
 ('приятно', 3),
 ('неустойчиво', 3),
 ('рекомендовать', 3),
 ('быть', 2),
 ('вставка', 2),
 ('качаться', 2),
 ('новоселье', 2),
 ('идеально', 2),
 ('заказ', 2),
 ('порадовать', 2),
 ('красивый', 2),
 (

## Тест суммаризатора

In [34]:
text = ". ".join(df_usefull["full_sentence"])
text

'Очень симпатично смотрится в интерьере. Упаковка целая, Пермскому картону респект) . Хорошое кашпо, советую!Обещали подарок положить,но к сожалению в коробке его не оказалось, не приятно!. Если ищете кашпо идеально белого цвета, то это не тот вариант.  Лёгкий пластик, активный ребенок или собака легко опрокинет, неустойчивое. Хотела два взять, пока только одно. Спасибо продавцу за бонус- земля для растения!!! Неожиданно и приятно!)))). Кашпо просто супер. Очень красиво смотрится. Буду заказывать еще на свои любимые растюшки.))) Грунт в подарок, что очень приятно. Вообщем всем рекомендую. . Лёгкое, но немного неустойчивое. Дно как-то выгнуто немного, шатается. Учитывая какой тяжёлый верх, внутрь нужно класть тяжёлые камни, думаю, для безопасности. Качество пластика нормальное. Горшок без отверстий, сделала 2шт. Сама, есть заготовки под дырочки, но сложно пробить, треснул немного на дне. В целом не плохой.. Заказала кашпо белого цвета, пришло бежевого((\nК самому товару нареканий нет. О

In [35]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [36]:
df_high_proba = df_usefull[df_usefull["proba"]> 0.7]
df_high_proba

,text,full_sentence,lemmas,class,prediction,proba
1030,Очень симпатично смотрится в интерьере.,Очень симпатично смотрится в интерьере. Упаков...,"[очень, симпатично, смотреться, интерьер]",Содержательные,Содержательные,0.981497
1815,", неустойчивое","Если ищете кашпо идеально белого цвета, то это...",[неустойчивый],Содержательные,Содержательные,0.746238
1914,Буду заказывать еще на свои любимые растюшки )...,Кашпо просто супер. Очень красиво смотрится. Б...,"[быть, заказывать, свой, любимый, растюшка, гр...",Содержательные,Содержательные,0.940924
1646,Дно как-то выгнуто немного.,"Лёгкое, но немного неустойчивое. Дно как-то вы...","[дно, както, выгнуть, немного]",Содержательные,Содержательные,0.821254
1226,Дизайн кашпо хорошо вписался в интерьер комнаты.,"Отличное кашпо, радует глаз. Дизайн кашпо хоро...","[дизайн, кашпо, вписаться, интерьер, комната]",Содержательные,Содержательные,0.989373
...,...,...,...,...,...,...
2233,Особенно порадовала цена 558 р !!!,"Отличный кашпо, хорошо упакован в коробку + зе...","[особенно, порадовать, цена, р]",Содержательные,Содержательные,0.877079
2756,Пакет с землёй всегда приятно получать подарки...,Рекомендую к покупке. Осталась очень довольна!...,"[пакет, земля, приятный, получать, подарок, ос...",Содержательные,Содержательные,0.890706
1035,Полностью вписался в интерьер.,Кашпо потрясающее. Полностью вписался в интерь...,"[полностью, вписаться, интерьер]",Содержательные,Содержательные,0.858989
1044,"Кашпо соответствует описанию, хорошее соотноше...","Кашпо соответствует описанию, хорошее соотноше...","[кашпо, соответствовать, описание, хороший, со...",Содержательные,Содержательные,0.877079


In [37]:
".".join(df_high_proba["full_sentence"])

'Очень симпатично смотрится в интерьере. Упаковка целая, Пермскому картону респект) .Если ищете кашпо идеально белого цвета, то это не тот вариант.  Лёгкий пластик, активный ребенок или собака легко опрокинет, неустойчивое. Хотела два взять, пока только одно. Спасибо продавцу за бонус- земля для растения!!! Неожиданно и приятно!)))).Кашпо просто супер. Очень красиво смотрится. Буду заказывать еще на свои любимые растюшки.))) Грунт в подарок, что очень приятно. Вообщем всем рекомендую. .Лёгкое, но немного неустойчивое. Дно как-то выгнуто немного, шатается. Учитывая какой тяжёлый верх, внутрь нужно класть тяжёлые камни, думаю, для безопасности. Качество пластика нормальное. Горшок без отверстий, сделала 2шт. Сама, есть заготовки под дырочки, но сложно пробить, треснул немного на дне. В целом не плохой..Отличное кашпо, радует глаз. Дизайн кашпо хорошо вписался в интерьер комнаты. Слишком легкое, поэтому для устойчивости положила утяжелитель на дно. .Кашпо пришло упаковано в картонную коро

In [38]:
input_ids = tokenizer(
        [" ".join(df_high_proba["text"])],
        max_length=2000,
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

output_ids = model.generate(input_ids=input_ids, no_repeat_ngram_size=4)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
summary

'Кашпо очень красиво вписалось в интерьер, очень хорошо вписался в интерьер. В подарок положили грунт для цветов, а также землю для растений.'

In [39]:
import torch
from transformers import AutoTokenizer, BertForTokenClassification

model_name = "IlyaGusev/rubert_ext_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
sep_token = tokenizer.sep_token
sep_token_id = tokenizer.sep_token_id

model = BertForTokenClassification.from_pretrained(model_name)

In [40]:
" ".join(df_high_proba["lemmas"].iloc[0])

'очень симпатично смотреться интерьер'

In [41]:
# def foo(data):
#     return " ".join(data)

In [42]:
# df_high_proba.assign(temp_lem=lambda x: foo(x["lemmas"]))

In [43]:
pd.concat([df_high_proba["text"].combine_first()], 
            axis=1)

TypeError: combine_first() missing 1 required positional argument: 'other'

In [ ]:
[row for row in df_high_proba]

['text', 'full_sentence', 'lemmas', 'class', 'prediction', 'proba']

In [ ]:
article_text = "..."
sentences = list(df_high_proba["text"])
article_text = sep_token.join(sentences)

In [ ]:
inputs = tokenizer(
    [article_text],
    max_length=500,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)
sep_mask = inputs["input_ids"][0] == sep_token_id

# Fix token_type_ids
current_token_type_id = 0 
for pos, input_id in enumerate(inputs["input_ids"][0]):
    inputs["token_type_ids"][0][pos] = current_token_type_id
    if input_id == sep_token_id:
        current_token_type_id = 1 - current_token_type_id

# Infer model
with torch.no_grad(): 
    outputs = model(**inputs) 
logits = outputs.logits[0, :, 1]

# Choose sentences 
logits = logits[sep_mask]
logits, indices = logits.sort(descending=True)
logits, indices = logits.cpu().tolist(), indices.cpu().tolist()
pairs = list(zip(logits, indices))
pairs = pairs[:3]
indices = list(sorted([idx for _, idx in pairs]))
summary = " ".join([sentences[idx] for idx in indices])
print(summary)

Дизайн кашпо хорошо вписался в интерьер комнаты. В подарок положили грунт к каждому кашпо. Земля в подарок.
